# Part 4: 결과 분석 및 시각화

이 노트북은 Part 3에서 이어집니다.

## 10. 전체 결과 로드 및 비교

In [ ]:
# 결과 로드
model_names = [
    'sequential_bam',
    'sequential_cae',
    'sequential_unet',
    'mtl_bam',
    'mtl_cae',
    'mtl_unet'
]

all_results = {}

print("Loading results...")
for model_name in model_names:
    with open(f'results/{model_name}_results.json', 'r') as f:
        all_results[model_name] = json.load(f)
    print(f"✓ {model_name}")

print("\n✓ All results loaded")

## 11. 결과 요약 테이블

In [ ]:
# 결과를 DataFrame으로 정리
results_data = []

for model_name in model_names:
    result = all_results[model_name]
    
    # 복원 성능
    if 'reconstruction' in result:
        recon_mse = result['reconstruction']['mse']
        recon_psnr = result['reconstruction']['psnr']
    elif 'restoration' in result:
        recon_mse = result['restoration']['mse']
        recon_psnr = result['restoration']['psnr']
    else:
        recon_mse = 0
        recon_psnr = 0
    
    # 분류 성능
    cls_acc = result['classification']['accuracy']
    
    results_data.append({
        'Model': model_name,
        'Type': 'Sequential' if 'sequential' in model_name else 'MTL',
        'Architecture': model_name.split('_')[-1].upper() if 'bam' in model_name else model_name.split('_')[-1].upper(),
        'Recon MSE': recon_mse,
        'Recon PSNR (dB)': recon_psnr,
        'Classification Acc': cls_acc
    })

results_df = pd.DataFrame(results_data)

print("\n" + "="*80)
print("COMPREHENSIVE RESULTS COMPARISON")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# CSV로 저장
results_df.to_csv('results/comprehensive_results.csv', index=False)
print("\n✓ Results saved to: results/comprehensive_results.csv")

## 12. 성능 비교 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# Figure 생성
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

# 1. Reconstruction MSE 비교
ax1 = axes[0, 0]
colors = ['#2E86AB' if 'sequential' in m else '#A23B72' for m in results_df['Model']]
ax1.barh(results_df['Model'], results_df['Recon MSE'], color=colors)
ax1.set_xlabel('MSE (lower is better)', fontsize=12)
ax1.set_title('Reconstruction MSE', fontsize=14, fontweight='bold')
ax1.invert_yaxis()

# 2. Reconstruction PSNR 비교
ax2 = axes[0, 1]
ax2.barh(results_df['Model'], results_df['Recon PSNR (dB)'], color=colors)
ax2.set_xlabel('PSNR (dB) (higher is better)', fontsize=12)
ax2.set_title('Reconstruction PSNR', fontsize=14, fontweight='bold')
ax2.invert_yaxis()

# 3. Classification Accuracy 비교
ax3 = axes[1, 0]
ax3.barh(results_df['Model'], results_df['Classification Acc'] * 100, color=colors)
ax3.set_xlabel('Accuracy (%) (higher is better)', fontsize=12)
ax3.set_title('Classification Accuracy', fontsize=14, fontweight='bold')
ax3.invert_yaxis()

# 4. Scatter: PSNR vs Accuracy
ax4 = axes[1, 1]
sequential_df = results_df[results_df['Type'] == 'Sequential']
mtl_df = results_df[results_df['Type'] == 'MTL']

ax4.scatter(sequential_df['Recon PSNR (dB)'], sequential_df['Classification Acc'] * 100,
           s=200, alpha=0.6, c='#2E86AB', label='Sequential', edgecolors='black', linewidth=2)
ax4.scatter(mtl_df['Recon PSNR (dB)'], mtl_df['Classification Acc'] * 100,
           s=200, alpha=0.6, c='#A23B72', label='MTL', edgecolors='black', linewidth=2)

# 레이블 추가
for idx, row in results_df.iterrows():
    arch = row['Architecture']
    ax4.annotate(arch, (row['Recon PSNR (dB)'], row['Classification Acc'] * 100),
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)

ax4.set_xlabel('Reconstruction PSNR (dB)', fontsize=12)
ax4.set_ylabel('Classification Accuracy (%)', fontsize=12)
ax4.set_title('PSNR vs Accuracy Trade-off', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved: results/performance_comparison.png")

## 13. 학습 곡선 비교

In [ ]:
# 학습 히스토리 로드
def load_history(model_name):
    """학습 히스토리 로드"""
    try:
        with open(f'history/{model_name}_history.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None

# MTL 모델 히스토리 로드
mtl_histories = {}
for model_name in ['mtl_bam', 'mtl_cae', 'mtl_unet']:
    hist = load_history(model_name)
    if hist:
        mtl_histories[model_name] = hist

# MTL 모델 학습 곡선 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('MTL Models Training Curves', fontsize=16, fontweight='bold')

colors = {'mtl_bam': '#2E86AB', 'mtl_cae': '#A23B72', 'mtl_unet': '#F18F01'}

# 1. Total Loss
ax = axes[0, 0]
for name, hist in mtl_histories.items():
    if 'loss' in hist:
        ax.plot(hist['loss'], label=f"{name} (train)", color=colors[name], alpha=0.7)
        ax.plot(hist['val_loss'], label=f"{name} (val)", color=colors[name], linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Total Loss')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 2. Reconstruction Loss
ax = axes[0, 1]
for name, hist in mtl_histories.items():
    recon_key = [k for k in hist.keys() if 'recon' in k.lower() and 'loss' in k.lower()][0]
    val_recon_key = 'val_' + recon_key
    ax.plot(hist[recon_key], label=f"{name} (train)", color=colors[name], alpha=0.7)
    if val_recon_key in hist:
        ax.plot(hist[val_recon_key], label=f"{name} (val)", color=colors[name], linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Reconstruction Loss')
ax.set_title('Reconstruction Loss')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 3. Classification Loss
ax = axes[0, 2]
for name, hist in mtl_histories.items():
    cls_key = [k for k in hist.keys() if ('cls' in k.lower() or 'classification' in k.lower()) and 'loss' in k.lower()][0]
    val_cls_key = 'val_' + cls_key
    ax.plot(hist[cls_key], label=f"{name} (train)", color=colors[name], alpha=0.7)
    if val_cls_key in hist:
        ax.plot(hist[val_cls_key], label=f"{name} (val)", color=colors[name], linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Classification Loss')
ax.set_title('Classification Loss')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 4. Reconstruction MAE
ax = axes[1, 0]
for name, hist in mtl_histories.items():
    mae_keys = [k for k in hist.keys() if 'mae' in k.lower() and 'val' not in k]
    if mae_keys:
        mae_key = mae_keys[0]
        val_mae_key = 'val_' + mae_key
        ax.plot(hist[mae_key], label=f"{name} (train)", color=colors[name], alpha=0.7)
        if val_mae_key in hist:
            ax.plot(hist[val_mae_key], label=f"{name} (val)", color=colors[name], linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('MAE')
ax.set_title('Reconstruction MAE')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 5. Classification Accuracy
ax = axes[1, 1]
for name, hist in mtl_histories.items():
    acc_keys = [k for k in hist.keys() if 'acc' in k.lower() and 'val' not in k and 'top' not in k]
    if acc_keys:
        acc_key = acc_keys[0]
        val_acc_key = 'val_' + acc_key
        ax.plot(hist[acc_key], label=f"{name} (train)", color=colors[name], alpha=0.7)
        if val_acc_key in hist:
            ax.plot(hist[val_acc_key], label=f"{name} (val)", color=colors[name], linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Classification Accuracy')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 6. Learning Rate
ax = axes[1, 2]
for name, hist in mtl_histories.items():
    if 'lr' in hist:
        ax.plot(hist['lr'], label=name, color=colors[name])
ax.set_xlabel('Epoch')
ax.set_ylabel('Learning Rate')
ax.set_title('Learning Rate Schedule')
ax.set_yscale('log')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/training_curves_mtl.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Training curves saved: results/training_curves_mtl.png")

## 14. 최종 요약 및 결론

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

# Best models
best_recon_model = results_df.loc[results_df['Recon PSNR (dB)'].idxmax()]
best_cls_model = results_df.loc[results_df['Classification Acc'].idxmax()]

print("\n[Best Reconstruction Performance]")
print(f"  Model: {best_recon_model['Model']}")
print(f"  PSNR: {best_recon_model['Recon PSNR (dB)']:.2f} dB")
print(f"  MSE: {best_recon_model['Recon MSE']:.6f}")

print("\n[Best Classification Performance]")
print(f"  Model: {best_cls_model['Model']}")
print(f"  Accuracy: {best_cls_model['Classification Acc']:.4f}")

# Sequential vs MTL comparison
print("\n[Sequential vs MTL Comparison]")
seq_results = results_df[results_df['Type'] == 'Sequential']
mtl_results = results_df[results_df['Type'] == 'MTL']

print(f"\nSequential Models:")
print(f"  Avg PSNR: {seq_results['Recon PSNR (dB)'].mean():.2f} dB")
print(f"  Avg Accuracy: {seq_results['Classification Acc'].mean():.4f}")

print(f"\nMTL Models:")
print(f"  Avg PSNR: {mtl_results['Recon PSNR (dB)'].mean():.2f} dB")
print(f"  Avg Accuracy: {mtl_results['Classification Acc'].mean():.4f}")

# Architecture comparison
print("\n[Architecture Comparison]")
for arch in ['BAM', 'CAE', 'UNET']:
    arch_results = results_df[results_df['Model'].str.contains(arch.lower())]
    if len(arch_results) > 0:
        print(f"\n{arch}:")
        print(f"  Avg PSNR: {arch_results['Recon PSNR (dB)'].mean():.2f} dB")
        print(f"  Avg Accuracy: {arch_results['Classification Acc'].mean():.4f}")

print("\n" + "="*80)
print("TRAINING COMPLETED SUCCESSFULLY!")
print("="*80)

print("\nSaved files:")
print("  - Weights: weights/")
print("  - Histories: history/")
print("  - Results: results/")
print("  - Logs: logs/")
print("\nAll models trained and evaluated successfully! 🎉")